In [161]:
import os
import cv2
import json
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

data_path ="./source/"
out_path = "./out.csv"
true_path = "./truth.csv"

class RawData:

    __all_train_filenames=None
    __train_data=None
    __begin_index=0
    __end_index=None

    @staticmethod
    def get_all_train_filenames():
        if RawData.__all_train_filenames is None:
            RawData.__all_train_filenames= os.listdir(data_path+"train"+"/annotations")
        return RawData.__all_train_filenames

    @staticmethod
    def get_train_data(begin=0,end=None):
        if RawData.__train_data is None or begin!=RawData.__begin_index or end!=RawData.__end_index:
            RawData.__begin_index=begin
            RawData.__end_index=end
            RawData.__train_data=[]
            RawData.get_all_train_filenames()
            for dataname in RawData.__all_train_filenames[begin:end]:
                f = open(data_path+"train/annotations/"+dataname, 'r')
                content = f.read()
                a = json.loads(content)
                f.close()
                a['image-name']=dataname[0:-5]+".jpg"
                RawData.__train_data.append(a)
        return RawData.__train_data

def to_logical(img):
    N = 48
    img = np.mean(img, 2)
    rows, cols = img.shape[:2]
    rc = rows // 2; cc = cols // 2
    img_crop = img[(rc-N):(rc+N), (cc-N*2):(cc+N*2)]
    if np.max(img_crop) - np.min(img_crop) < 40:
        img_crop = img
    b = (np.max(img_crop) + np.min(img_crop) + np.mean(img_crop)) / 3
    A = img > b
    if np.sum(A) < rows * cols / 2:
        return A
    else:
        return ~A

def to_num(label : str):
    labeled = ""
    for word in label:
        if word in "-0123456789.":
            labeled += word
        elif word == "(":
            if label.find("(") == 0:
                labeled += "-"
        elif word == ",":
            if label.find(".") == -1:
                l = label.split(",")
                if len(l) == 2 and len(l[1]) != 3:
                    labeled += "."
        elif word == "b":
            labeled += "6"
        elif word == "B":
            labeled += "8"
        elif word == "K":
            labeled += "000"
    try:
        return float(labeled)
    except ValueError:
        return None

def to_mask(img, all):
    rows = np.shape(img)[0]
    cols = np.shape(img)[1]
    mask = Image.new('L', (cols, rows), 0)
    for a in all:
        ImageDraw.Draw(mask).polygon(a, outline=1, fill=1)
    return np.array(mask) == 0

def find_points_in_vbar(img, r_end, c_begin, labels, mask):
    r_end = int(r_end)
    img_l = to_logical(img)
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    points = []
    for r in range(0, r_end):
        if np.sum(img_l[r, c_begin:]) > len(img_l[r, c_begin:]) * 0.8:
            img_l[r] = 0
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    for label in labels:
        c = np.argsort([label[1][1], label[2][1], label[3][1], label[4][1]])
        c = (label[c[0] + 1][0] + label[c[1] + 1][0]) // 2
        box = np.zeros(r_end, np.int32)
        for r in range(3, r_end):
            box[r] = np.sum(img_l[r-1:r+2, c-5:c+5])
        m = np.max(box)
        if m >= 8:
            for r in range(3, r_end):
                if box[r] > m * 0.9:
                    points.append(r)
                    break
        else:
            points.append(r_end - 5)
    return points

def find_points_in_hbar(img, r_end, c_begin, labels, mask):
    r_end = int(r_end)
    img_l = to_logical(img)
    cols = np.shape(img_l)[1]
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    points = []
    for c in range(c_begin, cols):
        if np.sum(img_l[:r_end, c]) > len(img_l[:r_end, c]) * 0.8:
            img_l[:, c] = 0
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    for label in labels:
        r = np.argsort([label[1][0], label[2][0], label[3][0], label[4][0]])
        r = (label[r[2] + 1][1] + label[r[3] + 1][1]) // 2
        box = np.zeros(cols, np.int32)
        for c in range(c_begin, cols - 3):
            box[c] = np.sum(img_l[r-5:r+5, c-1:c+2])
        m = np.max(box)
        if m >= 8:
            for c in reversed(range(c_begin, cols - 3)):
                if box[c] > m * 0.9:
                    points.append(c)
                    break
        else:
            points.append(c_begin + 5)
    return points

def find_points_in_dot(img, r_end, c_begin, labels, mask):
    r_end = int(r_end)
    img_l = to_logical(img)
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    points = []
    for r in range(0, r_end):
        if np.sum(img_l[r, c_begin:]) > len(img_l[r, c_begin:]) * 0.8:
            img_l[r] = 0
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    for label in labels:
        c = np.argsort([label[1][1], label[2][1], label[3][1], label[4][1]])
        c = (label[c[0] + 1][0] + label[c[1] + 1][0]) // 2
        box = np.zeros(r_end, np.int32)
        for r in range(3, r_end):
            box[r] = np.sum(img_l[r-1:r+2, c-5:c+5])
        m = np.max(box)
        if m >= 8:
            for r in range(3, r_end):
                if box[r] > m * 0.8:
                    points.append(r - 10)
                    break
        else:
            points.append(r_end - 5)
    return points

def find_points_in_line(img, r_end, c_begin, labels, mask):
    r_end = int(r_end)
    img_l = to_logical(img)
    cols = np.shape(img_l)[1]
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    points = []
    for r in range(0, r_end):
        if np.sum(img_l[r, c_begin:]) > len(img_l[r, c_begin:]) * 0.7:
            img_l[r] = 0
        points = []
    for c in range(c_begin, cols):
        if np.sum(img_l[:r_end, c]) > len(img_l[:r_end, c]) * 0.7:
            img_l[:, c] = 0
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    count = np.zeros(np.shape(img_l))
    for r in range(3, r_end):
        for c in range(c_begin, cols - 6):
            count[r, c] = np.sum(img_l[r-3:r+3, c-3:c+3])
    index = np.zeros(cols, np.int32)
    for c in range(c_begin, cols - 6):
        if np.max(count[:, c]) > 5:
            index[c] = np.argmax(count[:, c])
        else:
            index[c] = 0
    for c in range(c_begin, cols - 6):
        if abs(index[c - 1] - index[c + 1]) < 10:
            index[c] = (index[c - 1] + index[c + 1]) / 2
        elif abs(index[c - 2] - index[c + 2]) < 10:
            index[c] = (index[c - 2] + index[c + 2]) / 2
    # plt.plot(index)
    points = []
    for label in labels:
        c = np.argsort([label[1][1], label[2][1], label[3][1], label[4][1]])
        c = (label[c[0] + 1][0] + label[c[1] + 1][0]) // 2
        if index[c] != 0: points.append(index[c])
        elif index[c-1] != 0: points.append(index[c-1])
        elif index[c+1] != 0: points.append(index[c+1])
        elif index[c-2] != 0: points.append(index[c-2])
        elif index[c+2] != 0: points.append(index[c+2])
        elif index[c-3] != 0: points.append(index[c-3])
        elif index[c+3] != 0: points.append(index[c+3])
        elif index[c-4] != 0: points.append(index[c-4])
        elif index[c+4] != 0: points.append(index[c+4])
        elif index[c-5] != 0: points.append(index[c-5])
        elif index[c+5] != 0: points.append(index[c+5])
        else: points.append(np.nan)
    for i in range(len(points)):
        if not np.isnan(points[i]):
            break
    if i == len(points) - 1 and np.isnan(points[i]):
        return points
    for j in reversed(range(len(points))):
        if not np.isnan(points[j]):
            break
    for k in range(i, j + 1):
        if np.isnan(points[k]):
            points[k] = (points[i] + points[j]) / 2
    return points

def find_points_in_scat(img, r_begin, r_end, c_begin, mask):
    img_l = to_logical(img)
    # cv2.imwrite("step1.jpg", np.uint8(img_l) * 255)
    rows, cols = np.shape(img_l)
    r = np.zeros([rows, cols])
    c = np.zeros([rows, cols])
    img_l = img_l * mask
    # cv2.imwrite("step2.jpg", np.uint8(img_l) * 255)
    for i in range(r_begin, r_end):
        j = 0
        begin = 0
        now = False
        for j in range(c_begin, cols):
            if img_l[i, j] and not now:
                begin = j
                now = True
            elif not img_l[i, j] and now:
                r[i, (begin + j) // 2] = j - begin
                r[i, (begin + j) // 2 - 1] = j - begin
                r[i, (begin + j) // 2 + 1] = j - begin
                now = False
    for i in range(c_begin, cols):
        j = 0
        begin = 0
        now = False
        for j in range(r_begin, r_end):
            if img_l[j, i] and not now:
                begin = j
                now = True
            elif not img_l[j, i] and now:
                c[(begin + j) // 2, i] = j - begin
                c[(begin + j) // 2 - 1, i] = j - begin
                c[(begin + j) // 2 + 1, i] = j - begin
                now = False
    size = []
    lt = []
    for i in range(r_begin, r_end):
        for j in range(c_begin, cols):
            m = max(r[i, j], c[i, j])
            if img_l[i, j] and r[i, j] > 3 and c[i, j] > 3 and m < 20:
                size.append(min(r[i, j], c[i, j]))
                lt.append([i, j])
    l = []
    count = []
    size = np.max(size) ** 2 / 4.7
    for m in lt:
        flag = True
        for i in range(len(l)):
            s = (l[i][0] / count[i] - m[0]) ** 2 + (l[i][1] / count[i] - m[1]) ** 2
            if s < size:
                flag = False
                l[i][0] += m[0]
                l[i][1] += m[1]
                count[i] += 1
                break
        if flag:
            l.append(m)
            count.append(1)
    
    l = np.array(l, np.float64)
    count = np.array(count, np.float64)
    l[:, 0] = l[:, 0] / count
    l[:, 1] = l[:, 1] / count
    return l

def points_to_vals(points, labels, axis : int = 0):
    base = []
    val = []
    for label in labels:
        if to_num(label[0]) != None:
            base.append((label[1][axis] + label[2][axis] + label[3][axis] + label[4][axis]) / 4)
            val.append(to_num(label[0]))
    N = len(val)
    if N == 0:
        return points
    if N == 1:
        return (val[0] / base[0]) * points
    base = np.array(base)
    val = np.array(val)
    sor = np.argsort(base)
    base = base[sor]
    val = val[sor]
    flag = True
    while flag:
        flag = False
        for i in range(N):
            if i > 0 and i < N - 1:
                if val[i] > max(val[i-1], val[i+1]) or val[i] < min(val[i-1], val[i+1]):
                    N -= 1
                    val = np.delete(val, i)
                    base = np.delete(base, i)
                    flag = True
                    break
    
    N = len(val)
    w = (N * np.dot(val, base) - np.sum(val) * np.sum(base)) / (N * np.dot(base, base) - np.sum(base) ** 2)
    b = np.mean(val - w * base)
    points = w * np.array(points) + b
    return points

def points_to_vals_dot(points, labels):
    # 点图只需要寻找最接近的标签的 y 值
    N = len(labels)
    base = np.zeros(N)
    val = np.zeros(N)
    for i in range(N):
        base[i] = (labels[i][1][1] + labels[i][2][1] + labels[i][3][1] + labels[i][4][1]) / 4
        val[i] = to_num(labels[i][0])
    for i in range(len(points)):
        points[i] = val[np.argmin(np.abs(base - points[i]))]
    return points

def find_vals_in_vbar(img, xlabels, ylabels, all):
    l = len(ylabels)
    lx = len(xlabels)
    xs = np.zeros(l * 4, np.int32)
    xys = np.zeros(lx * 4, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
    for i in range(lx):
        xys[4 * i] = xlabels[i][1][1]
        xys[4 * i + 1] = xlabels[i][2][1]
        xys[4 * i + 2] = xlabels[i][3][1]
        xys[4 * i + 3] = xlabels[i][4][1]
    mask = to_mask(img, all)
    points = find_points_in_vbar(img, np.min(xys), np.max(xs), xlabels, mask)
    vals = points_to_vals(np.float64(points), ylabels, 1)
    return vals

def find_vals_in_hbar(img, xlabels, ylabels, all):
    l = len(ylabels)
    lx = len(xlabels)
    xs = np.zeros(l * 4, np.int32)
    xys = np.zeros(lx * 4, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
    for i in range(lx):
        xys[4 * i] = xlabels[i][1][1]
        xys[4 * i + 1] = xlabels[i][2][1]
        xys[4 * i + 2] = xlabels[i][3][1]
        xys[4 * i + 3] = xlabels[i][4][1]
    mask = to_mask(img, all)       
    points = find_points_in_hbar(img, np.min(xys), np.max(xs), ylabels, mask)
    vals = points_to_vals(np.float64(points), xlabels, 0)
    return vals

def find_vals_in_dot(img, xlabels, ylabels, all):
    l = len(ylabels)
    lx = len(xlabels)
    xs = np.zeros(l * 4, np.int32)
    xys = np.zeros(lx * 4, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
    for i in range(lx):
        xys[4 * i] = xlabels[i][1][1]
        xys[4 * i + 1] = xlabels[i][2][1]
        xys[4 * i + 2] = xlabels[i][3][1]
        xys[4 * i + 3] = xlabels[i][4][1]
    mask = to_mask(img, all)
    points = find_points_in_dot(img, np.min(xys), np.max(xs), xlabels, mask)
    vals = points_to_vals_dot(np.float64(points), ylabels)
    return vals

def find_vals_in_line(img, xlabels, ylabels, all):
    l = len(ylabels)
    xs = np.zeros(l * 4, np.int32)
    ys = np.zeros(l, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
        ys[i] = np.min([ylabels[i][1][1], ylabels[i][2][1], ylabels[i][3][1], ylabels[i][4][1]])
    mask = to_mask(img, all)   
    points = find_points_in_line(img, np.max(ys), np.max(xs), xlabels, mask)
    vals = points_to_vals(np.float64(points), ylabels, 1)
    return vals

def find_vals_in_scat(img, xlabels, ylabels, all):
    l = len(ylabels)
    xs = np.zeros(l * 4, np.int32)
    ys = np.zeros(l * 4, np.int32)
    for i in range(l):
        xs[4 * i] = ylabels[i][1][0]
        xs[4 * i + 1] = ylabels[i][2][0]
        xs[4 * i + 2] = ylabels[i][3][0]
        xs[4 * i + 3] = ylabels[i][4][0]
        ys[4 * i] = ylabels[i][1][1]
        ys[4 * i + 1] = ylabels[i][2][1]
        ys[4 * i + 2] = ylabels[i][3][1]
        ys[4 * i + 3] = ylabels[i][4][1]
    mask = to_mask(img, all)
    points = find_points_in_scat(img, np.min(ys), np.max(ys) + 1, np.max(xs), mask)
    vals = np.zeros(np.shape(points))
    # points 的 xy 是反的 遍历中img[i, j]的 i 代表 y
    vals[:, 0] = points_to_vals(np.float64(points[:, 1]), xlabels, 0)
    vals[:, 1] = points_to_vals(np.float64(points[:, 0]), ylabels, 1)
    sor = np.argsort(vals[:, 0])
    vals[:, 0] = vals[:, 0][sor]
    vals[:, 1] = vals[:, 1][sor]
    return vals

def find_vals_in(img, img_type : str, xlabels, ylabels, all):
    if len(xlabels) == 0 or len(ylabels) == 0:
        return []
    # labels 的每个元素为字符串 label name 与四个矩形顶点坐标 ['name', [x1, y1], [x2, y2], [x3, y3], [x4, y4]]
    if img_type == "vertical_bar":
        # 返回列表 对应按顺序每个xlabel的真实值
        return find_vals_in_vbar(img, xlabels, ylabels, all)
    elif img_type == "horizontal_bar":
        # 返回列表 对应按顺序每个ylabel的真实值
        return find_vals_in_hbar(img, xlabels, ylabels, all)
    elif img_type == "dot":
        # 返回列表 对应按顺序每个xlabel的真实值
        return find_vals_in_dot(img, xlabels, ylabels, all)
    elif img_type == "line":
        # 返回列表 对应按顺序每个xlabel的真实值 (无点时对应位置为 nan 用 np.isnan 判断)
        return find_vals_in_line(img, xlabels, ylabels, all)
    elif img_type == "scatter":
        # 返回二维列表 对应每个点的真实x值与y值
        return find_vals_in_scat(img, xlabels, ylabels, all)
    else:
        return None



In [162]:
def data_to_labels(data):
    type = data['chart-type']
    ress = data['data-series']
    labels = []
    all = []
    for text in data['text']:
        pol = text['polygon']
        all.append([pol['x0'], pol['y0'], pol['x1'], pol['y1'], pol['x2'], pol['y2'], pol['x3'], pol['y3']])
        if text['role'] == 'tick_label':  
            labels.append([text['text'], [pol['x0'], pol['y0']], [pol['x1'], pol['y1']], [pol['x2'], pol['y2']], [pol['x3'], pol['y3']]])
    xlabels = []
    ylabels = []
    if len(labels) < 2: return type, xlabels, ylabels, all
    if type == "vertical_bar":
        for label in labels:
            flag = False
            for res in ress:
                if label[0] == res['x']:
                    xlabels.append(label)
                    flag = True
                    break
            if not flag:
                ylabels.append(label)
        return type, xlabels, ylabels, all
    if type == "horizontal_bar":
        for label in labels:
            flag = False
            for res in ress:
                if label[0] == res['y']:
                    ylabels.append(label)
                    flag = True
                    break
            if not flag:
                xlabels.append(label)
        return type, xlabels, ylabels, all
    flag = 0
    if abs(labels[0][2][0] - labels[1][2][0]) < 8:
        t = labels[0][2][0]
        flag = 1
    if abs(labels[0][1][1] - labels[1][1][1]) < 8:
        t = labels[0][1][1]
        flag = 2
    elif abs(labels[0][2][1] - labels[1][2][1]) < 8:
        t = labels[0][2][1]
        flag = 4
    if flag == 1:
        for label in labels:
            if abs(label[2][0] - t) < 12:
                ylabels.append(label)
            else:
                xlabels.append(label)
    if flag == 2:
        for label in labels:
            if abs(label[1][1] - t) < 12:
                xlabels.append(label)
            else:
                ylabels.append(label)
    if flag == 4:
        for label in labels:
            if abs(label[2][1] - t) < 12:
                xlabels.append(label)
            else:
                ylabels.append(label)
    return type, xlabels, ylabels, all

from rapidfuzz.distance.Levenshtein import distance as levenshtein
from sklearn.metrics import r2_score

def sigmoid(x):
    return 2 - 2 / (1 + np.exp(-x))


def normalized_rmse(y_true, y_pred):
    # The argument to the sigmoid transform is equal to 
    # rmse(y_true, y_pred) / rmse(y_true, np.mean(y_true))
    return sigmoid((1 - r2_score(y_true, y_pred)) ** 0.5)


def normalized_levenshtein_score(y_true, y_pred):
    total_distance = np.sum([levenshtein(yt, yp) for yt, yp in zip(y_true, y_pred)])
    length_sum = np.sum([len(yt) for yt in y_true])
    return sigmoid(total_distance / length_sum)


def score_series(y_true, y_pred):
    if len(y_true) != len(y_pred):
        return 0
    if isinstance(y_true[0], str):
        res = normalized_levenshtein_score(y_true, y_pred)
    else:
        res = normalized_rmse(y_true, y_pred)
    if np.isnan(res):
        return 0
    else:
        return res

def score_all_ext(certain_type = ""):
    scores = [[], [], [], []]
    count = [0, 0, 0, 0]

    if certain_type == "" or certain_type == "vertical_bar":
        for dataname in os.listdir("./extracted/vertical_bar/"):
            count[0] += 1
            f = open("./source/train/annotations/"+ dataname[:-4] + ".json", 'r')
            content = f.read()
            a = json.loads(content)
            f.close()
            img_name = dataname[:-4] + ".jpg"
            img_type, xlabels, ylabels, all = data_to_labels(a)
            img = cv2.imread("./source/train/images/" + img_name)
            y = find_vals_in(img, img_type, xlabels, ylabels, all)
            x = []
            for xlabel in xlabels:
                x.append(xlabel[0])
            x_real = []
            y_real = []
            for now in a['data-series']:
                x_real.append(now['x'])
                if not np.isnan(now['y']):
                    y_real.append(now['y'])
            scores[0].append(score_series(x_real, x))
            scores[0].append(score_series(y_real, y))
            # if score_series(y_real, y) < 0.5 and score_series(y_real, y) > 0:
            #     print(img_name, score_series(y_real, y), y)
        scores[0] = np.mean(scores[0])
        if certain_type == "vertical_bar": return scores[0], count[0]
    
    if certain_type == "" or certain_type == "horizontal_bar":
        for dataname in os.listdir("./extracted/horizontal_bar/"):
            f = open("./source/train/annotations/"+ dataname[:-4] + ".json", 'r')
            content = f.read()
            a = json.loads(content)
            f.close()
            img_name = dataname[:-4] + ".jpg"
            img_type, xlabels, ylabels, all = data_to_labels(a)
            img = cv2.imread("./source/train/images/" + img_name)
            x = find_vals_in(img, img_type, xlabels, ylabels, all)
            y = []
            for ylabel in ylabels:
                y.append(ylabel[0])
            x_real = []
            y_real = []
            for now in a['data-series']:
                x_real.append(now['x'])
                y_real.append(now['y'])
            if score_series(y_real, y) > 0.5:
                count[1] += 1
                scores[1].append(score_series(x_real, x))
                scores[1].append(score_series(y_real, y))
                # if score_series(x_real, x) < 0.5:
                #     print(img_name, score_series(x_real, x), x)
        scores[1] = np.mean(scores[1])
        if certain_type == "horizontal_bar": return scores[1], count[1]
    
    if certain_type == "" or certain_type == "line":
        cnt = 0
        for dataname in os.listdir("./extracted/line/"):
            count[2] += 1
            f = open("./source/train/annotations/"+ dataname[:-4] + ".json", 'r')
            content = f.read()
            a = json.loads(content)
            f.close()
            img_name = dataname[:-4] + ".jpg"
            img_type, xlabels, ylabels, all = data_to_labels(a)
            img = cv2.imread("./source/train/images/" + img_name)
            val = find_vals_in(img, img_type, xlabels, ylabels, all)
            x = []
            y = []
            for i in range(len(xlabels)):
                if not np.isnan(val[i]):
                    x.append(xlabels[i][0])
                    y.append(val[i])
            x_real = []
            y_real = []
            for now in a['data-series']:
                x_real.append(now['x'])
                if not np.isnan(now['y']):
                    y_real.append(now['y'])
            scores[2].append(score_series(x_real, x))
            scores[2].append(score_series(y_real, y))
            if score_series(x_real, x) == 0:
                cnt += 1
        print(cnt)
        scores[2] = np.mean(scores[2])
        if certain_type == "line": return scores[2], count[2]

    if certain_type == "" or certain_type == "scatter":
        cnt1 = 0 
        cnt2 = 0
        for dataname in os.listdir("./extracted/scatter/"):
            count[3] += 1
            f = open("./source/train/annotations/"+ dataname[:-4] + ".json", 'r')
            content = f.read()
            a = json.loads(content)
            f.close()
            img_name = dataname[:-4] + ".jpg"
            img_type, xlabels, ylabels, all = data_to_labels(a)
            img = cv2.imread("./source/train/images/" + img_name)
            t = find_vals_in(img, img_type, xlabels, ylabels, all)
            x = t[:, 0]
            y = t[:, 1]
            x_real = []
            y_real = []
            for now in a['data-series']:
                if now['x'] is not None:
                    x_real.append(now['x'])
                if now['y'] is not None:
                    y_real.append(now['y'])
            scores[3].append(score_series(x_real, x))
            scores[3].append(score_series(y_real, y))
            if score_series(x_real, x) == 0:
                if len(x) > len(x_real): cnt1 += 1
                elif len(x) < len(x_real): cnt2 += 1
        print(cnt1, cnt2)
        scores[3] = np.mean(scores[3])
        if certain_type == "scatter": return scores[3], count[3]

    all_score = np.dot(scores, count) / np.sum(count)

    return all_score, scores, count

def scoring(begin = 0, end = 100, certain_type="", if_ext=True):
    train_data = RawData.get_train_data(begin, end)
    sc = [[], [], [], [], []]
    for data in train_data:
        img_type, xlabels, ylabels, all = data_to_labels(data)
        if certain_type != "":
            if img_type != certain_type:
                continue
        img = cv2.imread("./source/train/images/" + data['image-name'])
        if if_ext and data["source"] == "generated":
            continue
        try:
            vals = find_vals_in(img, img_type, xlabels, ylabels, all)
            x = []
            y = []
            if img_type == "vertical_bar":
                for xlabel in xlabels:
                    x.append(xlabel[0])
                y = vals
            elif img_type == "horizontal_bar":
                x = vals
                for ylabel in ylabels:
                    y.append(ylabel[0])
            elif img_type == "dot":
                for xlabel in xlabels:
                    x.append(xlabel[0])
                y = vals
            elif img_type == "line":
                for i in range(len(xlabels)):
                    if not np.isnan(vals[i]):
                        x.append(xlabels[i][0])
                        y.append(vals[i])
            elif img_type == "scatter":
                x = vals[:, 0]
                y = vals[:, 1]
            x_real = []
            y_real = []
            for now in data['data-series']:
                if now['x'] is not None:
                    x_real.append(now['x'])
                if now['y'] is not None:
                    y_real.append(now['y'])
            if img_type == "vertical_bar":
                sc[0].append(score_series(x_real, x))
                sc[0].append(score_series(y_real, y))
            elif img_type == "horizontal_bar":
                sc[1].append(score_series(x_real, x))
                sc[1].append(score_series(y_real, y))
            elif img_type == "dot":
                sc[2].append(score_series(x_real, x))
                sc[2].append(score_series(y_real, y))
            elif img_type == "line":
                sc[3].append(score_series(x_real, x))
                sc[3].append(score_series(y_real, y))
            elif img_type == "scatter":
                sc[4].append(score_series(x_real, x))
                sc[4].append(score_series(y_real, y))
        except:
            if img_type == "vertical_bar":
                sc[0].append(0)
                sc[0].append(0)
            elif img_type == "horizontal_bar":
                sc[1].append(0)
                sc[1].append(0)
            elif img_type == "dot":
                sc[2].append(0)
                sc[2].append(0)
            elif img_type == "line":
                sc[3].append(0)
                sc[3].append(0)
            elif img_type == "scatter":
                sc[4].append(0)
                sc[4].append(0)
    return [np.sum(sc[0]), np.sum(sc[1]), np.sum(sc[2]), np.sum(sc[3]), np.sum(sc[4])], [len(sc[0]), len(sc[1]), len(sc[2]), len(sc[3]), len(sc[4])]

def val_certain(img_name):
        f = open("./source/train/annotations/"+ img_name + ".json", 'r')
        content = f.read()
        a = json.loads(content)
        f.close()
        img_name = img_name + ".jpg"
        img_type, xlabels, ylabels, all = data_to_labels(a)
        img = cv2.imread("./source/train/images/" + img_name)
        return find_vals_in(img, img_type, xlabels, ylabels, all)

In [163]:
# print(score_all_ext(certain_type = ""))

In [164]:
val_certain("00ef8c0d39ba")

[[ 75.         405.        ]
 [ 92.66666667 373.        ]
 [108.         341.        ]
 [119.         309.        ]
 [126.         245.        ]
 [126.         277.        ]
 [137.         213.        ]
 [154.         181.        ]
 [185.         149.        ]
 [212.         117.        ]
 [220.          85.        ]]


array([[1.94966701e+03, 1.63043478e+00],
       [1.95969778e+03, 3.36956522e+00],
       [1.96972856e+03, 9.23913043e+00],
       [1.97975933e+03, 1.59782609e+01],
       [1.98979010e+03, 1.96739130e+01],
       [1.99982088e+03, 2.20652174e+01],
       [2.00985165e+03, 2.20652174e+01],
       [2.01988243e+03, 2.35869565e+01],
       [2.02991320e+03, 2.59782609e+01],
       [2.03994398e+03, 2.93115942e+01],
       [2.04997475e+03, 3.31521739e+01]])